## 0. Importação de pacotes

In [ ]:
!pip install scikit-learn==0.24

In [ ]:
import warnings
from collections import Counter
import math
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_log_error, accuracy_score
import xgboost as xgb
import lightgbm as lgb
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import GridSearchCV


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use(style='ggplot')
%matplotlib inline

In [ ]:
# utilis

def get_dict(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d

def get_first_gender(gender):
    d = get_dict(gender)
    if d != {}:
        return d[0]['name'] 
    return np.NaN

def count_values(col_list):
    d = get_dict(col_list)
    if d != {}:
        return len(d)
    return 0

def filmed_in_USA(prod_countries):
    d = get_dict(prod_countries)
    if d != {}:
        countries = [i['name'] for i in d]
        if 'United States of America' in countries:
            return 1    
    return 0

## 1. EDA
### Data load

In [ ]:
df = pd.read_csv('../input/tmdb-box-office-prediction/train.csv')

In [ ]:
df.head()

In [ ]:
df.loc[df['id'] == 16,'revenue'] = 192864          # Skinning
df.loc[df['id'] == 90,'budget'] = 30000000         # Sommersby          
df.loc[df['id'] == 118,'budget'] = 60000000        # Wild Hogs
df.loc[df['id'] == 149,'budget'] = 18000000        # Beethoven
df.loc[df['id'] == 313,'revenue'] = 12000000       # The Cookout 
df.loc[df['id'] == 451,'revenue'] = 12000000       # Chasing Liberty
df.loc[df['id'] == 464,'budget'] = 20000000        # Parenthood
df.loc[df['id'] == 470,'budget'] = 13000000        # The Karate Kid, Part II
df.loc[df['id'] == 513,'budget'] = 930000          # From Prada to Nada
df.loc[df['id'] == 797,'budget'] = 8000000         # Welcome to Dongmakgol
df.loc[df['id'] == 819,'budget'] = 90000000        # Alvin and the Chipmunks: The Road Chip
df.loc[df['id'] == 850,'budget'] = 90000000        # Modern Times
df.loc[df['id'] == 1112,'budget'] = 7500000        # An Officer and a Gentleman
df.loc[df['id'] == 1131,'budget'] = 4300000        # Smokey and the Bandit   
df.loc[df['id'] == 1359,'budget'] = 10000000       # Stir Crazy 
df.loc[df['id'] == 1542,'budget'] = 1              # All at Once
df.loc[df['id'] == 1542,'budget'] = 15800000       # Crocodile Dundee II
df.loc[df['id'] == 1571,'budget'] = 4000000        # Lady and the Tramp
df.loc[df['id'] == 1714,'budget'] = 46000000       # The Recruit
df.loc[df['id'] == 1721,'budget'] = 17500000       # Cocoon
df.loc[df['id'] == 1865,'revenue'] = 25000000      # Scooby-Doo 2: Monsters Unleashed
df.loc[df['id'] == 2268,'budget'] = 17500000       # Madea Goes to Jail budget
df.loc[df['id'] == 2491,'revenue'] = 6800000       # Never Talk to Strangers
df.loc[df['id'] == 2602,'budget'] = 31000000       # Mr. Holland's Opus
df.loc[df['id'] == 2612,'budget'] = 15000000       # Field of Dreams
df.loc[df['id'] == 2696,'budget'] = 10000000       # Nurse 3-D
df.loc[df['id'] == 2801,'budget'] = 10000000       # Fracture


In [ ]:
df.shape

# Belongs To Collection

In [ ]:
df.info()

In [ ]:
#profile = ProfileReport(df, title="EDA - Report")
#profile.to_file("eda_report_train.html")

# Cols to drop

In [ ]:
useless_cols = ['id', 'imdb_id', 'original_title', 'overview',
                'poster_path', 'tagline',
                'title', 'Keywords']

df.drop(columns=useless_cols, inplace=True)
df.shape

In [ ]:
df['genres'] = df['genres'].apply(get_first_gender)

In [ ]:
fig = px.box(df, y='revenue', x='genres')
fig.show()

In [ ]:
other_values = list(df['genres'].value_counts()[df['genres'].value_counts() < 50].index)
df['genres'].replace(to_replace=other_values, value='Other', inplace=True)

# pode melhorar isso aqui - agrupar melhor categorias de filmes

In [ ]:
fig = px.box(df, y='revenue', x='genres')
fig.show()

# Budget

In [ ]:
fig = px.histogram(df, x='budget')
fig.show()

# Homepage

In [ ]:
df['has_homepage'] = (~(df['homepage'].isna())).astype(int)


In [ ]:
df[['homepage', 'has_homepage']]

In [ ]:
fig = px.box(df, x='has_homepage', y='revenue', width=500)
fig.show()

## The scatter plot above seems interesting, this really shows that having a wepage for the movie is really affects its revenue

# Original Language

In [ ]:
df['original_language_en'] = (df['original_language']=='en').astype(int)

In [ ]:
fig = px.violin(df, x='original_language_en', y='revenue')
fig.show()

# Popularity

In [ ]:
fig = px.scatter(df, x='popularity', y='revenue', trendline='ols')

fig.show()

# Production Companies

### Most of films have 1-2 production companies, cometimes 3-4. But there are films with 10+ companies! Let's have a look at some of them.
### Not sure yet what i'm gonna do with these data

## I guess my best move for now to get the number of production companies, The most 50 common production companies

# Number of prod companies

In [ ]:
df['production_companies_number'] = df['production_companies'].apply(count_values)
df.drop(['production_companies'], axis=1, inplace=True)

In [ ]:
fig = px.box(df, x='production_companies_number', y='revenue')

fig.show()

# Production Country

In [ ]:
df['production_country_number'] = df['production_countries'].apply(count_values)

In [ ]:
df['crew_number'] = df['crew'].apply(count_values)
fig = px.scatter(df, x='crew_number', y='revenue', width=500)
fig.show()

In [ ]:
sns.catplot(x='production_country_number', y='revenue', data=df)

In [ ]:
df['production_in_USA'] = df['production_countries'].apply(filmed_in_USA)

In [ ]:
sns.catplot(x='production_in_USA', y='revenue', data=df)

### Some feature like is USA Production sould be useful here, let's see

In [ ]:
df.drop(['production_countries'], axis=1, inplace=True)

# Release Date

# Split the Release date to (Release Day, Release Month, Release Year)

In [ ]:
df[['release_month', 'release_day', 'release_year']] = df['release_date'].str.split('/', expand=True).replace(np.nan, -1).astype(int)

In [ ]:
df['release_year'] = (df['release_year'] < 20) * 100 + df['release_year'] + 1900

In [ ]:
df[['release_date', 'release_month', 'release_day', 'release_year']]

In [ ]:
px.line(df.groupby(by=['release_day']).median().reset_index(), x='release_day', y='revenue', width=500)

In [ ]:
df.drop(['release_date', 'release_day'], axis=1, inplace=True)



In [ ]:
df['release_year'] = (df['release_year'] < 20 * 100) + df['release_year']

In [ ]:
(df['release_year'] < 20) * 100 + df['release_year']

In [ ]:
px.box(df, x='release_month', y='revenue')

# Spoken Languages

## Let's generate some features like (how many languages have been spoken in the movie, is English has been spoken)

# Number of Spoken Languages

In [ ]:
df['spoken_languages_number'] = df['spoken_languages'].apply(count_values)
df.drop('spoken_languages', axis=1, inplace=True)

In [ ]:
sns.catplot(x='spoken_languages_number', y='revenue', data=df)

Clear pattern in the graph above

# Status

In [ ]:
df['status'].value_counts(dropna=False)

In [ ]:
df.drop(columns=['status'], inplace=True)

In [ ]:
df[['crew', 'crew_number']]

# Size of cast (number of actors)

In [ ]:
df['cast_number'] = df['cast'].apply(count_values)
df.drop(['cast'], axis=1, inplace=True)

In [ ]:
px.scatter(df, x='cast_number', y='revenue', trendline='ols')

### Belongs to colection

In [ ]:
df['belongs_to_collection'] = (~(df['belongs_to_collection'].isna())).astype(int)

In [ ]:
fig = px.box(df, x='belongs_to_collection', y='revenue')
fig.show()

## Cleaning

### Cleaning null



In [ ]:
df.isna().sum()
df.drop(columns=['crew', 'homepage'], inplace=True)

In [ ]:
(df[['runtime', 'revenue', 'budget']]==0).any(axis=1).sum()

In [ ]:
(df[['runtime', 'revenue', 'budget']]==0).any(axis=1).mean()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)
df.isna().sum()

In [ ]:
df[['genres']]

In [ ]:
pd.get_dummies(df[['genres']])

### Cleaning zeros


In [ ]:
cols_without_0 = ['runtime', 'revenue', 'budget']

for col in cols_without_0:
    df[col].replace(0, df[df[col]!=0][col].median(), inplace=True)

### Clip positive outliers

In [ ]:
cols_with_outliers = ['cast_number', 'spoken_languages_number', 'production_companies_number',
                      'popularity', 'budget']

#for col in cols_with_outliers:
#    limit = df[col].median() + (3*df[col].std())
#    df[col].clip(upper=limit, inplace=True)

In [ ]:
#df['revenue_log'] = np.log1p(df[['revenue']])

In [ ]:
fig = px.histogram(df, x='revenue', width=500)
fig.show()

In [ ]:
#fig = px.histogram(df, x='revenue_log', width=500)
#fig.show()

In [ ]:
fig = px.histogram(df, x='runtime', width=500)
fig.show()

In [ ]:
#limit_inf = df['runtime'].median() + (5*df['runtime'].std())
#df['runtime_cliped'] = df['runtime'].clip(upper=limit_inf)
#fig = px.histogram(df, x='runtime_cliped', width=500)
#fig.show()

### check data again

In [ ]:
#df.reset_index(drop=True, inplace=True)
#profile = ProfileReport(df, title="Final - Report")
#profile.to_file("Final_report.html")

# Model

In [ ]:
X = df.drop(columns='revenue')

y = np.log1p(df[['revenue']])

In [ ]:
X = pd.get_dummies(X, drop_first=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
def train_predict(model, train=(X_train, y_train), test=(X_test, y_test)):
    model.fit(train[0], train[1])
    y_train_pred = model.predict(train[0])

    train_error = np.sqrt(mean_squared_log_error(np.expm1(train[1]),
                                         np.expm1(y_train_pred)))
    
    model.predict(test[0])
    y_test_pred = model.predict(test[0])
    test_error = np.sqrt(mean_squared_log_error(np.expm1(test[1]),
                                         np.expm1(y_test_pred)))
    
    return (model, train_error, test_error)


In [ ]:
scaler = PowerTransformer()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lr = LinearRegression()
rfg = RandomForestRegressor(random_state=42)
svr = SVR()

In [ ]:
lr_results = train_predict(lr)
print(lr_results)

In [ ]:
rfg_results = train_predict(rfg)
print(rfg_results)

In [ ]:
svr_results = train_predict(svr)
print(svr_results)

# Optimize

In [ ]:
fig = px.histogram(df, x='budget')
fig.show()

# Tunning

In [ ]:
rfg = RandomForestRegressor(random_state=42)
parameters = {'bootstrap': [True, False],
     'max_depth': [10, 20, 30, 40,  None],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'n_estimators': [50, 100, 150, 200, 400]}

In [ ]:
grid = GridSearchCV(rfg, param_grid=parameters, scoring='neg_mean_absolute_error', verbose=100)

In [ ]:
grid_results = train_predict(grid)
print(grid_results)